In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
import random
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
drugs_cell_lines_ic50_df = pd.read_csv("data//drugs_cell_lines_ic50.csv")

In [ ]:
drugs_cell_lines_ic50_df.head()

In [ ]:
pubchem_drugs_smiles_df = pd.read_csv('data//drugs_smile_strings.csv')

In [ ]:
drugs_smiles_cell_lines_ic50_df = pd.merge(drugs_cell_lines_ic50_df, pubchem_drugs_smiles_df, 
                                             on = "drug_id")

In [ ]:
drugs_smiles_cell_lines_ic50_df = drugs_smiles_cell_lines_ic50_df[["drug_id", "Cancer_Cell_Line", "Smiles", "IC50"]]

In [ ]:
drugs_smiles_cell_lines_ic50_df.dtypes

In [ ]:
drugs_smiles_cell_lines_ic50_df["drug_id"] = drugs_smiles_cell_lines_ic50_df["drug_id"].astype(object)

In [ ]:
with open("data//drug_gcn_features.pickle", "rb") as f:
    dict_features = pickle.load(f)

In [ ]:
with open("data//drug_gcn_normalized_adj_mats.pickle", "rb") as f:
    dict_normalized_adj_mats = pickle.load(f)

In [ ]:
dualgcn_train = pd.read_csv("data//DualGCN_Embedding_train.csv")

In [ ]:
dualgcn_test = pd.read_csv("data//DualGCN_Embedding_test.csv")

In [ ]:
pubchem_to_drugs_df = pd.read_csv('data//1.Drug_listMon Jun 24 09_00_55 2019.csv')

In [ ]:
pubchem_to_drugs_df.head()

In [ ]:
pubchem_to_drugs_df = pubchem_to_drugs_df[["drug_id", "PubCHEM"]]

In [ ]:
pubchem_to_drugs_df.dtypes

In [ ]:
pubchem_to_drugs_df["PubCHEM"] = [val if str(val).isdigit() else np.nan for val in pubchem_to_drugs_df["PubCHEM"] ]

In [ ]:
pubchem_to_drugs_df = pubchem_to_drugs_df.dropna()

In [ ]:
pubchem_to_drugs_df.dtypes

In [ ]:
pubchem_to_drugs_df["drug_id"] = pubchem_to_drugs_df["drug_id"].astype(str)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(drugs_smiles_cell_lines_ic50_df.drop(["IC50"],1), drugs_smiles_cell_lines_ic50_df["IC50"].values, 
                                                     test_size = 0.20, random_state = 42)

In [ ]:
dualgcn_train["Drug_ID"] = dualgcn_train["Drug_ID"].astype(str)

In [ ]:
dualgcn_test["Drug_ID"] = dualgcn_test["Drug_ID"].astype(str)

In [ ]:
pubchem_to_drugs_df.dtypes

In [ ]:
dualgcn_train = pubchem_to_drugs_df.merge(dualgcn_train, left_on = ["PubCHEM"], right_on = ["Drug_ID"])

In [ ]:
dualgcn_train = dualgcn_train[['Cell_Line', 'drug_id']]

In [ ]:
dualgcn_test = pubchem_to_drugs_df.merge(dualgcn_test, left_on = ["PubCHEM"], right_on = ["Drug_ID"])

In [ ]:
dualgcn_test = dualgcn_test[['Cell_Line', 'drug_id']]

In [ ]:
dualgcn_train.dtypes

In [ ]:
x_train.dtypes

In [ ]:
x_train['drug_id'] = x_train['drug_id'].astype(str)

In [ ]:
x_valid['drug_id'] = x_valid['drug_id'].astype(str)

In [ ]:
x_train_valid_feats = pd.concat([x_train, x_valid], ignore_index = True)

In [ ]:
y_train_valid = pd.concat([pd.DataFrame(y_train.reshape(-1,1)), pd.DataFrame(y_valid.reshape(-1,1))], ignore_index = True)

In [ ]:
combo_train_valid = pd.concat([x_train_valid_feats, y_train_valid], 1)

In [ ]:
combo_train_valid.head()

In [ ]:
combo_train_valid.columns = ['drug_id', 'Cancer_Cell_Line', 'Smiles', 'IC50']

In [ ]:
# filter x_train x _valid here
x_y_train = combo_train_valid.merge(dualgcn_train, left_on = ['Cancer_Cell_Line','drug_id'], right_on = [ 'Cell_Line','drug_id'])

In [ ]:
x_y_train.head()

In [ ]:
x_y_test = combo_train_valid.merge(dualgcn_test, left_on = ['Cancer_Cell_Line','drug_id'], right_on = [ 'Cell_Line','drug_id'])

In [ ]:
x_train, x_valid, y_train, y_valid = x_y_train.drop(["IC50", 'Cell_Line'],1), x_y_test.drop(["IC50", 'Cell_Line'], 1), x_y_train["IC50"].values, x_y_test["IC50"].values

In [ ]:
train_gcn_feats = []
train_adj_list = []
for drug_id in x_train["drug_id"].values:
    train_gcn_feats.append(dict_features[drug_id])
    train_adj_list.append(dict_normalized_adj_mats[drug_id])

In [ ]:
valid_gcn_feats = []
valid_adj_list = []
for drug_id in x_valid["drug_id"].values:
    valid_gcn_feats.append(dict_features[drug_id])
    valid_adj_list.append(dict_normalized_adj_mats[drug_id])

In [ ]:
train_gcn_feats = np.array(train_gcn_feats).astype("float16")
valid_gcn_feats = np.array(valid_gcn_feats).astype("float16")

In [ ]:
train_adj_list = np.array(train_adj_list).astype("float16")
valid_adj_list = np.array(valid_adj_list).astype("float16")

In [ ]:
# load models
# omic models
cancer_copy_number_model = tf.keras.models.load_model("models//cancer_copy_number_model_no_norm_common")
cancer_cell_gen_expr_model = tf.keras.models.load_model("models//cancer_cell_gen_expr_model_no_norm_common")
cancer_cell_gen_methy_model = tf.keras.models.load_model("models//cancer_cell_gen_methy_model_no_norm")
cancer_cell_gen_mut_model = tf.keras.models.load_model("models//cancer_cell_gen_mut_model_no_norm")

In [ ]:
# load models
# drug models
pubchem_drugs_rdkit_model = tf.keras.models.load_model("models//pubchem_drugs_rdkit_model_no_norm")

In [ ]:
std = StandardScaler()

In [ ]:
# extract drug features - does not seem like these are used in the network
drug_features_train = pubchem_drugs_rdkit_model(x_train["drug_id"].values).numpy().astype("float32")
drug_features_valid = pubchem_drugs_rdkit_model(x_valid["drug_id"].values).numpy().astype("float32")

# drug_features_train = pubchem_drugs_rdkit_model(x_train["drug_id"].values)
# drug_features_valid = pubchem_drugs_rdkit_model(x_valid["drug_id"].values)

In [ ]:
np.isinf(drug_features_train).sum()

In [ ]:
drug_features_train = std.fit_transform(drug_features_train)

In [ ]:
drug_features_valid = std.transform(drug_features_valid)

In [ ]:
# extract copy number features
omics_copy_number_train = cancer_copy_number_model(x_train["Cancer_Cell_Line"].values).numpy().astype("float16")
omics_copy_number_valid = cancer_copy_number_model(x_valid["Cancer_Cell_Line"].values).numpy().astype("float16")

In [ ]:
# extract gen expr features
omics_gen_expr_train = cancer_cell_gen_expr_model(x_train["Cancer_Cell_Line"].values).numpy().astype("float16")
omics_gen_expr_valid = cancer_cell_gen_expr_model(x_valid["Cancer_Cell_Line"].values).numpy().astype("float16")

In [ ]:
omics_gen_copy_number_gen_expr_train = np.concatenate([np.expand_dims(omics_copy_number_train, -1),
                                                      np.expand_dims(omics_gen_expr_train, -1)], axis = -1)

In [ ]:
omics_gen_copy_number_gen_expr_valid = np.concatenate([np.expand_dims(omics_copy_number_valid, -1),
                                                      np.expand_dims(omics_gen_expr_valid, -1)], axis = -1)

In [ ]:
# extract gen methylation features
omics_gen_methyl_train = cancer_cell_gen_methy_model(x_train["Cancer_Cell_Line"].values).numpy().astype("float16")
omics_gen_methyl_valid = cancer_cell_gen_methy_model(x_valid["Cancer_Cell_Line"].values).numpy().astype("float16")

In [ ]:
# extract gen mutation features
with tf.device('/cpu:0'):
    omics_gen_mut_train = cancer_cell_gen_mut_model.predict(x_train["Cancer_Cell_Line"].values, verbose = 1, batch_size = 256).astype("float16")
    omics_gen_mut_valid = cancer_cell_gen_mut_model.predict(x_valid["Cancer_Cell_Line"].values, verbose = 1, batch_size = 256).astype("float16")

In [ ]:
smile_strings_train = x_train["Smiles"].values.reshape(-1,1)
smile_strings_valid = x_valid["Smiles"].values.reshape(-1,1)

In [ ]:
selected_info_common_cell_lines = "data//cellline_list.txt"
selected_info_common_genes = "data//gene_list.txt"

In [ ]:
PPI_file = "data/PPI_network.txt"

In [ ]:
with open(selected_info_common_cell_lines) as f:
    common_cell_lines = [item.strip() for item in f.readlines()]

In [ ]:
with open("data//common_genes.pickle", "rb") as f:
    common_genes = pickle.load(f)

In [ ]:
idx_dic={}
for index, item in enumerate(common_genes):
    idx_dic[item] = index

In [ ]:
ppi_adj_info = [[] for item in common_genes] 

In [ ]:
# will return for each gene what other gene is connected - PPIs
ppi_adj_info = [[] for item in common_genes] 
for line in open(PPI_file).readlines():
    gene1,gene2 = line.split('\t')[0],line.split('\t')[1]
    if (gene1 in common_genes) & (gene2 in common_genes):
        if idx_dic[gene1]<=idx_dic[gene2]:
            ppi_adj_info[idx_dic[gene1]].append(idx_dic[gene2])
            ppi_adj_info[idx_dic[gene2]].append(idx_dic[gene1])

In [ ]:
def CelllineGraphAdjNorm(ppi_adj_info,common_genes = common_genes):
    # with open(selected_info_common_genes) as f:
    #     common_genes = [item.strip() for item in f.readlines()]
    nb_nodes = len(common_genes)
    adj_mat = np.zeros((nb_nodes,nb_nodes),dtype='float32')
    # print(adj_mat.shape)
    for i in range(len(ppi_adj_info)):
        # print(i)
        nodes = ppi_adj_info[i]
        for each in nodes:
            adj_mat[i,each] = 1

    # for checking if two sparse matrices are the same
    assert np.allclose(adj_mat,adj_mat.T)
    norm_adj = NormalizeAdj(adj_mat)
    return norm_adj 

In [ ]:
def NormalizeAdj(adj):
    adj = adj + np.eye(adj.shape[0])
    d = sp.diags(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0).toarray()
    a_norm = adj.dot(d).transpose().dot(d)
    return a_norm

In [ ]:
import scipy.sparse as sp

In [ ]:
ppi_adj = CelllineGraphAdjNorm(ppi_adj_info,common_genes)

In [ ]:
ppi_adj = np.expand_dims(ppi_adj,0)

In [ ]:
omics_gen_copy_number_gen_expr_train_new = (ppi_adj@omics_gen_copy_number_gen_expr_train)
omics_gen_copy_number_gen_expr_valid_new = (ppi_adj@omics_gen_copy_number_gen_expr_valid)

In [ ]:
copy_number_train = omics_gen_copy_number_gen_expr_train_new[:,:,0:1]
copy_number_valid = omics_gen_copy_number_gen_expr_valid_new[:,:,0:1]

In [ ]:
gene_expr_train = omics_gen_copy_number_gen_expr_train_new[:,:,1:2]
gene_expr_valid = omics_gen_copy_number_gen_expr_valid_new[:,:,1:2]

In [ ]:
valid_items = [[ valid_gcn_feats, valid_adj_list,
                           copy_number_valid, gene_expr_valid,
                           omics_gen_methyl_valid, omics_gen_mut_valid], y_valid]

In [ ]:
# import the existing trained model
simpleGCN_original_model = tf.keras.models.load_model("models//simple_gcn_new_splits")

In [ ]:
simpleGCN_original_model.summary()

In [ ]:
# get the layer before the last layer
before_pred = simpleGCN_original_model.layers[-2].output

In [ ]:
# add the prediction layer
pred_layer = tf.keras.layers.Dense(1, name = 'last_dense')
pred_out = pred_layer(before_pred)

In [ ]:
new_gcn_model = tf.keras.models.Model(inputs = [simpleGCN_original_model.input], outputs = [pred_out])

In [ ]:
new_gcn_model.summary()

In [ ]:
new_gcn_model.compile(loss = tf.keras.losses.MeanSquaredError(), 
                    optimizer = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), 
                    metrics = [tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
# First the data - let's use a train set of 6k - then the rest will be test here (from the remaining values, use 3k for training the stacker, 1k for validation of it, and the rest ~7k as the final test data). 
# drug feats
new_train_gcn_feats = valid_gcn_feats[:10000, :,:]
new_test_gcn_feats = valid_gcn_feats[10000:, :,:]
print(new_train_gcn_feats.shape, new_test_gcn_feats.shape)

# drug adj info
new_train_adj_list = valid_adj_list[:10000, :,:]
new_test_adj_list = valid_adj_list[10000:, :,:]
print(new_train_adj_list.shape, new_test_adj_list.shape)

# cnv
new_omics_cna_train = copy_number_valid[:10000, :]
new_omics_cna_test = copy_number_valid[10000:,:]
print(new_omics_cna_train.shape, new_omics_cna_test.shape)

# expression
new_omics_expr_train = gene_expr_valid[:10000, :]
new_omics_expr_test = gene_expr_valid[10000:,:]
print(new_omics_cna_train.shape, new_omics_cna_test.shape)

# y
new_y_train = y_valid[:10000,]
new_y_test = y_valid[10000:,]
print(new_y_train.shape, new_y_test.shape)

In [ ]:
%%time
history = new_gcn_model.fit([new_train_gcn_feats, new_train_adj_list, new_omics_cna_train, new_omics_expr_train], new_y_train, 
                         
          batch_size = 64, epochs = 1000, verbose = 1,
                         
          validation_split = 0.1,
                         

        callbacks = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 20, restore_best_weights=True,
                                                       mode = "min"), 
         validation_batch_size = 512, shuffle = True)

In [ ]:
new_gcn_model.save("models//gcn_all_feats_new_splits")

In [ ]:
val_preds = new_gcn_model.predict([new_test_gcn_feats, new_test_adj_list, new_omics_cna_test ,new_omics_expr_test])

In [ ]:
preds_data = pd.DataFrame(np.hstack((new_y_test.reshape(-1,1), val_preds)), columns = ['True_y', 'Predicted_y_gcn'])

In [ ]:
preds_data.to_csv("data/gcn_all_feats_data_new_split.csv", index = False)